- Get financial details and marketcap with RapidAPI, for all 507 tickers. 
- Merge marketcap data with sim_snp_esg_full data. 


In [8]:
import pandas as pd
import os 
import random
import requests
import datetime
import gc 

In [9]:
os.getcwd()

'C:\\Users\\User\\@Code-ML\\[Project]-ESG_responsible_investment\\work\\notebook\\ETL'

### Getting ticker list from data 

In [4]:
api_key = os.environ.get('x-rapidapi-key')

In [ ]:
snp500_df = pd.read_csv("../../data/transform/dim_snp_esg_full_16-Jan-2022.csv")
display(snp500_df.shape)
snp500_df.head()

In [161]:
snp500_list = snp500_df['ticker'].tolist()
display(len(snp500_list))
random.sample(snp500_list, 10)

507

['PLD', 'SLB', 'MDLZ', 'GPN', 'AVY', 'JNPR', 'ITW', 'CCI', 'HSY', 'LLY']

### Testing the API 

In [5]:
def financial_data_scrapper_test(one_ticker):
    error_tickers = []
    url = "https://yh-finance.p.rapidapi.com/stock/v2/get-summary"
    headers = {
        'x-rapidapi-host': "yh-finance.p.rapidapi.com",
        'x-rapidapi-key': api_key
        }
    scraped_df = pd.DataFrame()

    querystring = {"symbol":one_ticker,"region":"US"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    data_row = pd.json_normalize(response.json())
    scraped_df = pd.concat([scraped_df, data_row], axis=0).reset_index(drop=True)
    return scraped_df

In [6]:
df2 = financial_data_scrapper_test("AAPL")

In [7]:
df2

,symbol,defaultKeyStatistics.enterpriseToRevenue.raw,defaultKeyStatistics.enterpriseToRevenue.fmt,defaultKeyStatistics.profitMargins.raw,defaultKeyStatistics.profitMargins.fmt,defaultKeyStatistics.enterpriseToEbitda.raw,defaultKeyStatistics.enterpriseToEbitda.fmt,defaultKeyStatistics.52WeekChange.raw,defaultKeyStatistics.52WeekChange.fmt,defaultKeyStatistics.forwardEps.raw,...,esgScores.environmentScore.raw,esgScores.environmentScore.fmt,esgScores.governancePercentile,esgScores.militaryContract,upgradeDowngradeHistory.history,upgradeDowngradeHistory.maxAge,pageViews.shortTermTrend,pageViews.midTermTrend,pageViews.longTermTrend,pageViews.maxAge
0,AAPL,7.817,7.82,0.25882,25.88%,23.785,23.78,0.286071,28.61%,6.19,...,0.29,0.3,None,False,"[{'epochGradeDate': 1642161976, 'firm': 'Loop ...",86400,DOWN,UP,UP,1


### Get data 
Using Rapid API to get data. API limit rate is 500 calls.  

Wonder why append doesnt work.. But pd.concat works fine.. 

In [147]:

def financial_data_scrapper(dl_list):
    error_tickers = []
    url = "https://yh-finance.p.rapidapi.com/stock/v2/get-summary"
    headers = {
        'x-rapidapi-host': "yh-finance.p.rapidapi.com",
        'x-rapidapi-key': api_key
        }
    scraped_df = pd.DataFrame()
    marketcap_df = pd.DataFrame()
    marketcap_tuple_list = []

    for ticker in dl_list: 
        try:
            querystring = {"symbol":ticker,"region":"US"}
            response = requests.request("GET", url, headers=headers, params=querystring)
#             data_row = pd.json_normalize(response.json())
#             scraped_df.append(data_row)
            data_row = pd.json_normalize(response.json())
            scraped_df = pd.concat([scraped_df, data_row], axis=0).reset_index(drop=True)
            print(f'Successfully processed {ticker} at {datetime.datetime.now() }!')
            print(f'DataFrame size is {scraped_df.shape}.\n')
            
            marketcap_tuple = (scraped_df['symbol'].iloc[-1], scraped_df['price.marketCap.raw'].iloc[-1] )                
            print("Marketcap: ",marketcap_tuple)
            marketcap_tuple_list.append(marketcap_tuple)
            
        except KeyError:
            error_tickers.append(ticker)
            print(f'This ticker {ticker} is not found.\n')
            continue
    return scraped_df, marketcap_tuple_list

In [148]:
#Getting the reamaining data 
second_scraped_df, second_marketcap_tuple_list = financial_data_scrapper(snp500_list[500:])         

Successfully processed XRAY at 2022-01-17 16:54:33.816570!
DataFrame size is (1, 449).

Marketcap:  ('XRAY', 11791662080)
Successfully processed XYL at 2022-01-17 16:54:36.125030!
DataFrame size is (2, 449).

Marketcap:  ('XYL', 20113448960)
Successfully processed YUM at 2022-01-17 16:54:38.493570!
DataFrame size is (3, 449).

Marketcap:  ('YUM', 37245476864)
Successfully processed ZBH at 2022-01-17 16:54:40.901587!
DataFrame size is (4, 449).

Marketcap:  ('ZBH', 26610700288)
Successfully processed ZBRA at 2022-01-17 16:54:43.218884!
DataFrame size is (5, 450).

Marketcap:  ('ZBRA', 28217112576)
Successfully processed ZION at 2022-01-17 16:54:45.825517!
DataFrame size is (6, 450).

Marketcap:  ('ZION', 11054111744)
Successfully processed ZTS at 2022-01-17 16:54:48.226250!
DataFrame size is (7, 450).

Marketcap:  ('ZTS', 97549115392)


### Full scraped data

In [102]:
first_scraped_df = scraped_df[:500]

In [154]:
full_scraped_df = pd.concat([first_scraped_df, second_scraped_df])
#full_scraped_df.to_csv('full_financial_details.csv')

### Marketcap data 

In [ ]:
first_marketcap_tuple_list = marketcap_tuple_list[:500]

In [159]:
full_marketcap_tuple_list = first_marketcap_tuple_list +second_marketcap_tuple_list
df_marketcap = pd.DataFrame(full_marketcap_tuple_list, columns =['Ticker', 'Marketcap'])
display(df_marketcap.shape)
df_marketcap.head()

(507, 2)

,Ticker,Marketcap
0,A,4.369350e+10
1,AAL,1.197253e+10
2,AAP,1.494031e+10
3,AAPL,2.826995e+12
4,ABBV,2.402018e+11


In [160]:
#df_marketcap.to_csv('marketcap_data.csv')

In [6]:
import pandas as pd
pd.set_option('display.max_rows', None)
df = pd.read_csv("../../data/raw/full_financial_details.csv")
df.T

,0,1,2,3,4,5,6,7,8,9,...,497,498,499,500,501,502,503,504,505,506
Unnamed: 0,0,1,2,3,4,5,6,7,8,9,...,497,498,499,0,1,2,3,4,5,6
symbol,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
defaultKeyStatistics.enterpriseToRevenue.raw,7.126,1.801,1.628,7.964,5.596,0.159,13.416,5.567,4.126,15.628,...,4.706,13.64,1.498,3.235,4.079,7.524,4.212,5.313,0.349,13.263
defaultKeyStatistics.enterpriseToRevenue.fmt,7.13,1.8,1.63,7.96,5.6,0.16,13.42,5.57,4.13,15.63,...,4.71,13.64,1.5,3.23,4.08,7.52,4.21,5.31,0.35,13.26
defaultKeyStatistics.profitMargins.raw,0.19148,-0.13229,0.05895,0.25882,0.13656,0.0072,0.15319,0.17122,0.11533,0.30548,...,0.12056,0.23055,-0.02435,0.09847,0.08808,0.24499,0.10392,0.15454,0.36889,0.26024
defaultKeyStatistics.profitMargins.fmt,19.15%,-13.23%,5.89%,25.88%,13.66%,0.72%,15.32%,17.12%,11.53%,30.55%,...,12.06%,23.06%,-2.43%,9.85%,8.81%,24.50%,10.39%,15.45%,36.89%,26.02%
defaultKeyStatistics.enterpriseToEbitda.raw,26.104,-9.273,14.886,24.231,10.699,10.681,45.327,19.048,24.556,37.493,...,13.909,46.107,10.796,13.676,24.421,21.297,13.951,24.556,NaN,32.21
defaultKeyStatistics.enterpriseToEbitda.fmt,26.1,-9.27,14.89,24.23,10.7,10.68,45.33,19.05,24.56,37.49,...,13.91,46.11,10.8,13.68,24.42,21.3,13.95,24.56,NaN,32.21
defaultKeyStatistics.52WeekChange.raw,0.136082,0.157796,0.443982,0.353908,0.211178,0.315942,-0.104312,0.138492,0.381176,0.140416,...,0.065832,0.42401,0.47154,-0.075103,0.069825,0.183164,-0.203527,0.301005,0.432191,0.268254
defaultKeyStatistics.52WeekChange.fmt,13.61%,15.78%,44.40%,35.39%,21.12%,31.59%,-10.43%,13.85%,38.12%,14.04%,...,6.58%,42.40%,47.15%,-7.51%,6.98%,18.32%,-20.35%,30.10%,43.22%,26.83%


## Merge marketcap data with sim_snp_esg_full data

In [19]:
c = pd.read_csv("../../data/transform/snp_500_marketcap_data_2022-01-17.csv" )
df_snp_esg_full = pd.read_csv("../../data/transform/dim_snp_esg_full_16-Jan-2022.csv")
display(df_marketcap.shape)
df_snp_esg_full.shape

(507, 2)

(507, 5)

In [ ]:
#Rename column
df_marketcap['ticker']= df_marketcap['Ticker'] 
del df_marketcap['Ticker']

In [32]:
df_snp_esg_full = df_snp_esg_full.merge(df_marketcap, on='ticker', how='inner')
df_snp_esg_full.head()

,ticker,company_name,gics_sector,gics_sub_industry,is_esg,Marketcap
0,A,Agilent Technologies,Health Care,Health Care Equipment,1,4.369350e+10
1,AAL,American Airlines Group,Industrials,Airlines,1,1.197253e+10
2,AAP,Advance Auto Parts,Consumer Discretionary,Automotive Retail,0,1.494031e+10
3,AAPL,Apple,Information Technology,"Technology Hardware, Storage & Peripherals",1,2.826995e+12
4,ABBV,AbbVie,Health Care,Pharmaceuticals,1,2.402018e+11


In [31]:
df_snp_esg_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 507 entries, 0 to 506
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ticker             507 non-null    object
 1   company_name       507 non-null    object
 2   gics_sector        507 non-null    object
 3   gics_sub_industry  507 non-null    object
 4   is_esg             507 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 19.9+ KB


In [33]:
df_snp_esg_full.to_csv('dim_snp_esg_full_marketcap_2022-01-19.csv')